In [1]:
import requests
import unidecode
import unicodedata
import json
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
page = requests.get('https://apps.qoo-app.com/en/app-card/9038')
page_source = BeautifulSoup(page.text, "html.parser")
page_source

<!DOCTYPE html>

<html lang="en" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="en" name="language"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0" name="viewport"/>
<meta content="webkit" name="renderer"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>[Cards] Project Sekai Colorful Stage Feat. Hatsune Miku | Japanese - QooApp Game Store</title>
<meta content="[Cards] Project Sekai Colorful Stage Feat. Hatsune Miku | Japanese" name="description"/>
<meta content="website" property="og:type"/>
<meta content="[Cards] Project Sekai Colorful Stage Feat. Hatsune Miku | Japanese - QooApp Game Store" property="og:title"/>
<meta content="https://apps.qoo-app.com/en/app-card/9038" property="og:url"/>
<meta content="[Cards] Project Sekai Colorful Stage Feat. Hatsune Miku | Japanese" property="og:description"/>
<meta content="QooApp" property="og:site_name"/>
<meta content="https://play-lh.googleusercontent.com/oPca

In [3]:
posts_list = page_source.findAll("div", class_="qoo-post-item")
posts_list

[<div class="qoo-post-item" data-id="296022" data-type="game_card">
 <a class="icon qoo-egg-view" data-hover-popup="user" href="https://user.qoo-app.com/en/41737965">
 <div class="slot" data-args="avatar=https%3A%2F%2Fimg.qoo-static.com%2Favatar%2F202012%2F03%2FikhjlenjzM4KEm0qjTd1mCER.jpeg%3Fm%3D1606996083" data-slot="AVATAR_PANEL"></div>
 </a>
 <header class="qoo-clearfix" data-user-id="41737965">
 <cite class="name">
 <a class="artist" data-hover-popup="user" href="https://user.qoo-app.com/en/41737965">S</a>
 <a href="https://apps.qoo-app.com/en/app/9038">Project Sekai Colorful Stage Feat. Hatsune Miku | Japanese</a>
 </cite>
 <time class="time">2021-12-18 15:04:24</time>
 </header>
 <section class="text-view">
 <p>
                                                 Game Card created, 1 cards shared
                                             </p>
 <table>
 <tr>
 <td>Player Name:</td>
 <td>白沢</td>
 </tr>
 </table>
 <p>
 <strong>Introduction:</strong>
                         终于出了一张限定

In [4]:
def normalize(s):
    new_s = unicodedata.normalize("NFKD", s.lower())
    #new_s = unidecode.unidecode(new_s)
    new_s = new_s.replace("4", "a")
    new_s = new_s.replace("3", "e")
    
    return new_s

In [15]:
post_info_list = []
for post in posts_list:
    post_info = {"user": "", "link": "", "char_name": "", "guild_name": "", "desc": ""}
    
    #print(post.select('a.icon.qoo-egg-view')[0]["href"])
    post_info["link"] = 'https://user.qoo-app.com/en/card/' + post["data-id"]
    td = post.findAll('td')
    if len(td)>0 :
        post_info["char_name"] = normalize(td[1].get_text(strip=True))
    if len(td)>2 :
        post_info["guild_name"] = normalize(td[3].get_text(strip=True))
    
    desc = post.findAll('p')
    if len(desc) > 1:
        post_info["desc"] = normalize(desc[1].get_text(strip=True))
    
    post_info_list.append(post_info)
print(post_info_list)

[{'user': '', 'link': 'https://user.qoo-app.com/en/card/296022', 'char_name': '白沢', 'guild_name': '', 'desc': 'introduction:终于出了一张限定...'}, {'user': '', 'link': 'https://user.qoo-app.com/en/card/296011', 'char_name': 'kaito', 'guild_name': '', 'desc': 'introduction:有三张欲望大哥的初始,私心想换有傲慢铃的初始'}, {'user': '', 'link': 'https://user.qoo-app.com/en/card/295987', 'char_name': '。', 'guild_name': '', 'desc': 'introduction:哇哇哇\\( ö )/'}, {'user': '', 'link': 'https://user.qoo-app.com/en/card/295882', 'char_name': 'len', 'guild_name': '', 'desc': 'introduction:หลังจากที่บ่นไปว่าส่งตัวซ้ํามาทําไม ล่าสุดได้เลนมาแล้ว55555555'}, {'user': '', 'link': 'https://user.qoo-app.com/en/card/295875', 'char_name': 'kiritani haruka', 'guild_name': '', 'desc': 'introduction:di spam ama haruka wkwk'}]


In [16]:
with open("blackwordlist.json", 'r', encoding='utf-8-sig') as file:
    word_list = json.load(file)["word_list"]

In [18]:
violating_post = []
for post in post_info_list:
    if any(word in post["char_name"] for word in word_list) or any(word in post["desc"] for word in word_list) or any(word in post["guild_name"] for word in word_list) :
        violating_post.append(post)

[]

In [19]:
reply = ""
for post in violating_post:
    reply = reply + post["link"] + '\n'
print(reply)